To run this example locally, [install Ploomber](https://ploomber.readthedocs.io/en/latest/get-started/install.html) and execute: `ploomber examples -n cookbook/python-load`

To start a free, hosted JupyterLab: [![binder-logo](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/ploomber/binder-env/main?urlpath=git-pull%3Frepo%3Dhttps%253A%252F%252Fgithub.com%252Fploomber%252Fprojects%26urlpath%3Dlab%252Ftree%252Fprojects%252Fcookbook/python-load%252FREADME.ipynb%26branch%3Dmaster)

Found an issue? [Let us know.](https://github.com/ploomber/projects/issues/new?title=cookbook/python-load%20issue)

Have questions? [Ask us anything on Slack.](https://ploomber.io/community/)


# Loading pipeline.yaml in Python

<!-- start description -->
Load pipeline.yaml file in a Python session to customize ininitialization.
<!-- end description -->

## Loading the pipeline

Create a function that loads it and returns the DAG object:

```python
# Content of pipeline.py
from ploomber.spec import DAGSpec
from ploomber import with_env


@with_env('env.yaml')
# NOTE: you may add other params to the function, they'll show up in the cli
def make(env):
    dag = DAGSpec('pipeline.yaml', env=dict(env)).to_dag()
    # NOTE: return the DAG, do not call any methods
    return dag

```

*Note:* If your pipeline isn't using an `env.yaml` file, simply remove the decorator, and the `env` argument in the function.

## Command-line interface

The CLI will work just as if you were using the `pipeline.yaml` file directly, pass the dotted path to the function in the `--entry-point/-e` argument:

In [1]:
%%sh
ploomber status -e pipeline.make

name         Last run      Outdated?    Product       Doc (short)    Location
-----------  ------------  -----------  ------------  -------------  ------------
create_file  Has not been  Source code  File('produc  None           /Users/Edu/d
             run                        ts/number/so                 ev/projects-
                                        me_file.txt'                 ploomber/coo
                                        )                            kbook/python
                                                                     -load/tasks.
                                                                     py:4


/Users/Edu/dev/ploomber/src/ploomber/spec/dagspec.py:336: UserWarning: The following placeholders are declared in the environment but unused in the spec: {'user', 'cwd', 'now', 'here', 'root'}
  warnings.warn('The following placeholders are declared in the '
100%|██████████| 1/1 [00:00<00:00, 1114.91it/s]


In [2]:
%%sh
ploomber build -e pipeline.make

name         Ran?      Elapsed (s)    Percentage
-----------  ------  -------------  ------------
create_file  True         0.000616           100


/Users/Edu/dev/ploomber/src/ploomber/spec/dagspec.py:336: UserWarning: The following placeholders are declared in the environment but unused in the spec: {'user', 'root', 'now', 'here', 'cwd'}
  warnings.warn('The following placeholders are declared in the '
Building task 'create_file': 100%|██████████| 1/1 [00:03<00:00,  3.35s/it]
